In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Define emojis for gestures
gesture_emojis = {
    "Thumbs Up": "👍",
    "Peace": "✌️",
    "Pointing Up": "☝️",
    "Fist": "✊",
    "Waving Hand": "👋",
    "Love You": "🤟",
    "Unknown": "❓"
}

def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    ba = np.array([a.x, a.y, a.z]) - np.array([b.x, b.y, b.z])
    bc = np.array([c.x, c.y, c.z]) - np.array([b.x, b.y, b.z])
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

def get_fingers_extended(landmarks):
    """Check which fingers are extended."""
    return [landmarks[tip].y < landmarks[pip].y - 0.05 for tip, pip in zip([8, 12, 16, 20], [6, 10, 14, 18])]

def classify_gesture(thumb_angle, fingers_extended):
    """Classify the gesture based on thumb angle and finger extensions."""
    if fingers_extended == [1, 1, 0, 0]: return "Peace"
    if fingers_extended == [1, 0, 0, 0]: return "Pointing Up"
    if all(not f for f in fingers_extended): return "Thumbs Up" if thumb_angle > 100 else "Fist"
    if all(fingers_extended): return "Waving Hand"
    if fingers_extended == [1, 0, 0, 1]: return "Love You"
    return "Unknown"

# Camera setup
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = hand_landmarks.landmark
            thumb_angle = calculate_angle(landmarks[4], landmarks[2], landmarks[0])
            fingers_extended = get_fingers_extended(landmarks)

            gesture = classify_gesture(thumb_angle, fingers_extended)
            cv2.putText(image, f"{gesture} {gesture_emojis.get(gesture, '')}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('MediaPipe Hands', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()